In [1]:
import numpy as np
import pandas as pd
import os.path
import shutil
from datetime import datetime, timedelta
from dateutil.rrule import rrule, MONTHLY

In [2]:
generated_path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\checked\\'
beals_path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\beals\\'

In [3]:
def add_property_ids_to_beals(dfB):
    dfB.loc[(dfB.ID.str.match('.*Flat 1[,]? 321.*',case=False)==True),'Property'] = 'F1321LON'
    dfB.loc[(dfB.ID.str.match('.*Flat 1[,]? 169.*',case=False)==True),'Property'] = 'F1169FAW'
    dfB.loc[(dfB.ID.str.match('.*Flat 1[,]? 171.*',case=False)==True),'Property'] = 'F1171FAW'
    dfB.loc[(dfB.ID.str.match('.*Flat 2[,]? 171.*',case=False)==True),'Property'] = 'F2171FAW'
    dfB.loc[(dfB.ID.str.match('.*196a.*',case=False)==True),'Property'] = '196AKIN'
    dfB.loc[(dfB.ID.str.match('.*Shop[,]? 196.*',case=False)==True),'Property'] = 'SHOP196KIN'
    dfB.loc[(dfB.ID.str.match('.*Flat 3[,]? 163.*',case=False)==True),'Property'] = 'F3163FRA'
    dfB.loc[(dfB.ID.str.match('.*Flat 6[,]? 8.*',case=False)==True),'Property'] = 'F68ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 7[,]? 8.*',case=False)==True),'Property'] = 'F78ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 5[,]? 12[-]?14.*',case=False)==True),'Property'] = 'F51214ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 7[,]? 12[-]?14.*',case=False)==True),'Property'] = 'F71214ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 10[,]? 12[-]?14.*',case=False)==True),'Property'] = 'F101214ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 14[,]? 12[-]?14.*',case=False)==True),'Property'] = 'F141214ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 16[,]? 12[-]?14.*',case=False)==True),'Property'] = 'F161214ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 6[,]? 16[-]?18.*',case=False)==True),'Property'] = 'F61618ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 11[,]? 16[-]?18.*',case=False)==True),'Property'] = 'F111618ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 12[,]? 16[-]?18.*',case=False)==True),'Property'] = 'F121618ALH'
    dfB.loc[(dfB.ID.str.match('.*Flat 16[,]? 16[-]?18.*',case=False)==True),'Property'] = 'F161618ALH'
    return dfB

def load_bank(start, end):
    start_date=datetime.strptime(start, '%Y-%m-%d')
    end_date=datetime.strptime(end, '%Y-%m-%d')  
    dates = [dt for dt in rrule(MONTHLY, dtstart=start_date, until=end_date)]

    dfAll=pd.DataFrame()
    dfAll=pd.DataFrame(columns=['Account','Amount','Subcategory','Memo','Property','Description','Cat','Subcat'])

    for date in dates:
        dateStr= date.strftime("%b").upper() + date.strftime("%Y")
        input_file=generated_path + dateStr + '_codedAndCategorised.csv'
        if not os.path.isfile(input_file):
            print('Warning missing file: ' + input_file)
            continue
        dfTemp=pd.read_csv(input_file,  parse_dates=['Date'], dayfirst=True)
        dfAll=pd.concat([dfAll,dfTemp])
        dfAll=dfAll[['Date','Account','Amount','Subcategory','Memo','Property','Description','Cat','Subcat']]
    return dfAll

def load_beals(start, end):
    start_date=datetime.strptime(start, '%Y-%m-%d')
    end_date=datetime.strptime(end, '%Y-%m-%d')  
    dates = [dt for dt in rrule(MONTHLY, dtstart=start_date, until=end_date)]

    dfBAll=pd.DataFrame()

    for date in dates:
        dateStr= date.strftime("%b").upper() + date.strftime("%Y")
        dfB=pd.read_csv(beals_path + 'Beals_' + dateStr + '.csv', parse_dates=['Date Created'], dayfirst=True)
        dfB.insert(0,'Date', dfB['Date Created'].dt.date)
        dfB['Date'] =  pd.to_datetime(dfB.Date)
        if 'Transaction Type' in dfB.columns:
            dfB.rename(columns={'Transaction Type': 'ID'}, inplace=True)
        dfB=dfB.sort_values(by=['ID','Date Created'])
        dfB[['ID','Activity I D','Date Created','Item Description','Document Type','Document Number','Debit Amount','Credit Amount']]
        dfB['Property']=dfB['ID']
        dfB=dfB.reset_index()
        dfB.drop('index', axis=1, inplace=True)
        dfBAll=pd.concat([dfBAll,dfB])
    return dfBAll

def reconcile_beals_with_bank(df_bank, df_beals):
    lookforward = 5
    errorcount = 0
    if 'ReconciledBeals' not in df_bank.columns:
        df_bank['ReconciledBeals']=0
    if 'ReconciledBank' not in df_beals.columns:
        df_beals['ReconciledBank']=0
    if 'ReconciledBankAccount' not in df_beals.columns:
        df_beals['ReconciledBankAccount']=''
        
    for index, row in dfB[dfB['Item Description'].str.contains("Amount payable to", na=False)].sort_index().iterrows():
        print('Reconciling: ' + str(row.Date) + ' ' + row['Item Description'] + ' ' + str(row['Debit Amount']))

        startdate = row.Date 
        enddate = row.Date + timedelta(days=lookforward)
        
        rec=df_bank.loc[(df_bank['Property']==row.Property)&(df_bank['Amount']==row['Debit Amount'])&(df_bank.Date>=startdate)&(df_bank.Date<=enddate)&(df_bank.ReconciledBeals!=1)]
        if not rec.empty:
            print('\tWith '+ str(rec.iloc[0].Date)+ ' ' + rec.Memo.iloc[0] + ' ' + str(rec.Amount.iloc[0]))
            df_bank.at[rec.iloc[0].name, 'ReconciledBeals']=1
            df_beals.at[index, 'ReconciledBank']=1
            df_beals.at[index, 'ReconciledAccount']=rec.iloc[0].Account
        else:
            print('CANNOT RECONCILE')
            errorcount = errorcount + 1

    print('Errors=' +  str(errorcount))   

### Load Data

In [4]:
# Reconciliation period
start='2022-01-01'
end='2022-08-31'
dfAll=load_bank(start,end)
dfB=load_beals(start,end)

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### Allocate Property Codes to Beals data

- check codes are allocated correctly

In [5]:
dfB=add_property_ids_to_beals(dfB)
dfB[(dfB.Property.isnull())]

,Activity I D,Credit Amount,Date,Date Created,Debit Amount,Document Number,Document Type,Grouped Amount,ID,ID Check,Item Description,Lettings Charge Group I D,Property,Shared Html,Unnamed: 0,Unnamed: 8,Uploaded Invoice


### Reconcile Bank payments against Beals

Iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.

Mark reconciled data in bank and reconciled Beals payments with 1, mark Beals transactions with account they were paid into.

In [6]:
dfAll.ReconciledBeals=0
dfB.ReconciledBank=0
dfB.ReconciledBankAccount=''
reconcile_beals_with_bank(dfAll,dfB)

Reconciling: 2022-01-05 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
	With 2022-01-07 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
Reconciling: 2022-04-05 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
	With 2022-04-07 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
Reconciling: 2022-05-04 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
	With 2022-05-06 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
Reconciling: 2022-06-06 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
	With 2022-06-08 00:00:00 BEALS ESTATE AGENT    	196A KINGSTON ROAD BG 470.32
Reconciling: 2022-07-05 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
	With 2022-07-07 00:00:00 BEALS ESTATE AGENT    	196A KINGSTON ROAD BG 470.32
Reconciling: 2022-08-02 00:00:00 Amount payable to Mr Mathew Tucker - 

	With 2022-01-11 00:00:00 BEALS ESTATE AGENT    FLAT 1412-14 ALHAM BGC 452.0
Reconciling: 2022-06-01 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 207.42
	With 2022-06-06 00:00:00 BEALS ESTATE AGENT    	FLAT 1216-18 ALHAM BG 207.42
Reconciling: 2022-05-10 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 171.84
	With 2022-05-12 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 171.84
Reconciling: 2022-04-26 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 321 London Road, Portsmouth 446.0
	With 2022-04-28 00:00:00 BEALS ESTATE AGENT    FLAT 1321 LONDON R BGC 446.0
Reconciling: 2022-08-23 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 320.15
	With 2022-08-25 00:00:00 BEALS ESTATE AGENT FLAT 1216-18 ALHAM  320.15
Reconciling: 2022-07-26 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 321 London Road, Portsmouth 446.0
	With 2022-07-28 00:00:00 BEALS ESTATE A

	With 2022-02-17 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 286.16
Reconciling: 2022-01-25 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 311.8
	With 2022-01-27 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 311.8
Reconciling: 2022-06-28 00:00:00 Amount payable to Mr Mathew Tucker - Flat 2, 171 Fawcett Road, Southsea 241.39
	With 2022-06-30 00:00:00 BEALS ESTATE AGENT    	FLAT 2171 FAWCETT BGC 241.39
Reconciling: 2022-08-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 369.0
	With 2022-08-10 00:00:00 BEALS ESTATE AGENT FLAT 68 ALHAMBRA R  369.0
Reconciling: 2022-04-12 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 416.75
	With 2022-04-14 00:00:00 BEALS ESTATE AGENT    FLAT 616-18 ALHAMB BGC 416.75
Reconciling: 2022-03-12 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 416.75
	With 2022-03-16 00:00:00 BEALS ESTATE AGENT    FL

### Unreconciled Beals payments not received in our bank

- payments Beals said they made but that were not received
- check dates are not at end of month and so might have been paid in next months data

In [10]:
dfB[['Date','ID','Item Description','Debit Amount','Credit Amount','ReconciledAccount']][(dfB['Item Description'].str.contains("Amount payable to", na=False))&(dfB['ReconciledBank']==0)]

,Date,ID,Item Description,Debit Amount,Credit Amount,ReconciledAccount


### Unreconciled Bank payments

- payments received in our bank, not in Beals data

In [8]:
dfAll[(dfAll.Memo.str.contains("BEALS ESTATE AGENT", na=False))&(dfAll['ReconciledBeals']==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Cat,Subcat,ReconciledBeals
115,2022-02-10,20-74-09 60458872,420.96,DIRECTDEP,BEALS ESTATE AGENT FLAT 1169 FAWCETT BGC,F1169FAW,NaN,BealsRent,NaN,0
45,2022-06-06,20-53-97 30728691,196.16,Counter Credit,BEALS ESTATE AGENT \tFLAT 312-14 ALHAMB BG,F31214ALH,NaN,BealsRent,NaN,0
49,2022-06-06,20-53-97 30728691,328.84,Counter Credit,BEALS ESTATE AGENT \tFLAT 312-14 ALHAMB BG,F31214ALH,NaN,BealsRent,NaN,0


### Output Reconciliation Data

In [9]:
dfAll.to_csv('BankReconciliationCheck.csv')
dfB.to_csv('BealsReconciliationCheck.csv')